In [1]:
#! pip install -U langchain-openai
#! pip install lark
#! pip install langchain-community langchain-core
#!pip install -U langchain-chroma

In [1]:
# Load File Directory
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import TextLoader

# Document Splitting
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

# Embeddings og vectorstores
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.output_parsers import PydanticOutputParser
from langchain_chroma import Chroma

# llm søgning
from lark import Lark
from langchain.llms import OpenAI
from langchain_openai import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

# komprimering
import warnings
from langchain.retrievers.document_compressors import EmbeddingsFilter

# Search
import os
import openai
import sys




# Make Langchain and vectorstore run  

In [8]:
OPENAI_API_KEY = 'your-api-key-goes-here'

client = openai.OpenAI(api_key= OPENAI_API_KEY)

def get_completion(prompt, model="gpt-4o"): # eller "gpt-4" ,   "gpt-3.5-turbo", "gpt-4o", "gpt-3.5-turbo-0125"
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content

# Load the vectorstore

In [11]:
persist_directory = 'docs Light in French Literature/chroma/'

In [17]:
embedding = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding
)

In [19]:
# Antal af dokumenter i samlingen
print(vectordb._collection.count())

41103


# Similarity Search
Get the results that are most similar to your question.

we use 'k' to deside how many results we want.

In [24]:
question = 'Find examples where lighted or illuminated spaces tributes to enthusiastic and loving feeling.'

In [ ]:
docs = vectordb.similarity_search(question,k=4)

count = 0
for i in docs:
    count = count + 1
    print ('Example: '+ str(count))
    print ('\n')
    print (str(i.page_content).replace('\n',' ').replace('Ãˆ','È').replace('Ãª', 'ê').replace('â€™','’')\
                              .replace('Å“', 'œ').replace('Ã©', 'é').replace('Ã®','î')\
                                .replace('Ã‰', 'É').replace('Ã¨', 'è').replace('Ã', 'à')\
                                 .replace('à¢', 'â').replace('â€¦', '…').replace('à”', 'Ô')\
                                  .replace('à¹','ù'))
    
    source = str(i.metadata).replace('}','').replace('.txt','').split('\\')[-1]
    print (f"Source: {source}")
    print ('*'*50)
    print ('\n')

# Maximum marginal relevance¶
It is not always that you are interested in getting the results that are most similar to your question. Maximum marginal relevance (MMR) is about ensuring diversity in the search results, and achieving both relevance for the query and diversity among the results.

The idea is that you send a question to your vector database and get some answers back. We use "fetch_k=" to determine how many responses we get back. It is based on semantic similarity alone. From there, we work further with those answers and find the most relevant based on semantic similarity and based on difference. After this we choose "k=", which are the answers that will be output.

_docs_mmr = vectordb.max_marginal_relevance_search(question,k=3, fetch_k=10)_

In [ ]:
question = 'Find examples where lighted or illuminated spaces tributes to enthusiastic and loving feeling.'

In [ ]:
docs_mmr = vectordb.max_marginal_relevance_search(question,k=3, fetch_k=10)

count = 0
for i in docs_mmr:
    count = count + 1
    print ('Example: '+ str(count))
    print ('\n')
    print (str(i.page_content).replace('Ãˆ','È').replace('Ãª', 'ê').replace('â€™','’')\
                              .replace('Å“', 'œ').replace('Ã©', 'é').replace('Ã®','î')\
                                .replace('Ã‰', 'É').replace('Ã¨', 'è').replace('Ã', 'à')\
                                 .replace('à¢', 'â').replace('â€¦', '…').replace('à”', 'Ô')\
                                  .replace('à¹','ù'))
    
    source = str(i.metadata).replace('}','').replace('.txt','').split('\\')[-1]
    print (f"Source: {source}")
    print ('*'*50)
    print ('\n')